In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error

import tensorflow as tf
#import tensorflow_probability as tfp

In [ ]:
train_dataset = pd.read_csv('./train.csv')
plt.figure(figsize=(12,6))
corr_matrix = train_dataset.drop(columns=['id']).corr()
sns.heatmap(corr_matrix, annot=True)

In [ ]:
plt.figure(figsize=(15, 7))
plt.subplot(221)
sns.scatterplot(data=train_dataset, x='allelectrons_Average', y='atomicweight_Average', hue='Hardness')
plt.subplot(222)
sns.scatterplot(data=train_dataset, x='allelectrons_Average', y='R_vdw_element_Average', hue='Hardness')
plt.subplot(223)
sns.scatterplot(data=train_dataset, x='allelectrons_Average', y='R_cov_element_Average', hue='Hardness')
plt.subplot(224)
sns.scatterplot(data=train_dataset, x='allelectrons_Average', y='density_Average', hue='Hardness')

In [ ]:
features = ['allelectrons_Total', 'density_Total', 'allelectrons_Average',
            'val_e_Average', 'atomicweight_Average', 'ionenergy_Average',
            'el_neg_chi_Average', 'R_vdw_element_Average', 'R_cov_element_Average',
            'zaratio_Average', 'density_Average', 'Hardness']

In [ ]:
X = train_dataset[features].drop(columns='Hardness')
y = train_dataset.Hardness

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
model_pre = LGBMRegressor()
model_pre.fit(X, y)
X_new = X.copy()
X_new['Hardness_pred'] = model_pre.predict(X)

In [ ]:
def loss_fn(y_true, y_pred):
    return tfp.stats.percentile(tf.abs(y_true - y_pred), q=50)

def metric_fn(y_true, y_pred):
    return tfp.stats.percentile(tf.abs(y_true - y_pred), q=100) - tfp.stats.percentile(tf.abs(y_true - y_pred), q=0)

In [ ]:
callbacks_list = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=2, mode='min',restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, min_lr=0.00001),
    tf.keras.callbacks.TerminateOnNaN()
]

In [ ]:
def create_model():

    input_layer = tf.keras.Input(shape=(len(features), ))
    x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(input_layer)
    x = tf.keras.layers.Dense(16, activation='relu', kernel_initializer='glorot_uniform')(x)
    #x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(x)
    x = tf.keras.layers.Dense(32, activation='relu', kernel_initializer='glorot_uniform')(x)
    #x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(x)
    output_layer = tf.keras.layers.Dense(1)(x)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

    model.compile(optimizer=tf.keras.optimizers.Adam(0.013, beta_1=0.5),
                  loss=loss_fn,
                  metrics=metric_fn)

    return model

model = create_model()
history = model.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=100,
                    batch_size = 64,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list,
                    validation_split=0.1)

In [ ]:
sample_submission = pd.read_csv('./sample_submission.csv')
test_dataset = pd.read_csv('./test.csv')

In [ ]:
test_dataset['Hardness_pred'] = model_pre.predict(test_dataset.astype('float32').drop(columns='id'))
test_dataset["Hardness"] = model.predict(test_dataset.astype('float32').drop(columns='id'))

sample_submission = test_dataset[['id', "Hardness"]]
sample_submission.to_csv("submission.csv", index=False)

In [ ]:
plt.figure(figsize=(15,4))

plt.subplot(121)
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Training and validation losses')
plt.legend()

In [ ]:
sample_submission

In [ ]:
from matplotlib import pyplot as plt
sample_submission.plot(kind='scatter', x='id', y='Hardness', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
sample_submission['Hardness'].plot(kind='hist', bins=20, title='Hardness')
plt.gca().spines[['top', 'right',]].set_visible(False)